In [1]:
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import tensorflow as tf
tfk = tf.keras
tf.keras.backend.set_floatx("float64")
import tensorflow_probability as tfp
tfd = tfp.distributions
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from keras import backend as K

%config InlineBackend.figure_format = 'retina'
import pandas as pd
%matplotlib inline
from keras import activations, initializers
from keras.layers import Layer
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, Activation, LeakyReLU
from tensorflow_probability.python.layers import DistributionLambda
import tensorflow_probability as tfp
tfd = tfp.distributions

%matplotlib inline

Using TensorFlow backend.


In [2]:
noise = 1
def neg_log_likelihood_with_dist(y_true, y_pred, sigma = noise):
    return -tf.reduce_mean(y_pred.log_prob(y_true))

In [3]:
def mixture_prior_params(sigma_1, sigma_2, pi, return_sigma=False):
    params = K.variable([sigma_1, sigma_2, pi], name='mixture_prior_params')
    sigma = np.sqrt(pi * sigma_1 ** 2 + (1 - pi) * sigma_2 ** 2)
    return params, sigma
  
def log_mixture_prior_prob(w):
    comp_1_dist = tf.distributions.Normal(0.0, prior_params[0])
    comp_2_dist = tf.distributions.Normal(0.0, prior_params[1])
    comp_1_weight = prior_params[2]
    return K.log(comp_1_weight * comp_1_dist.prob(w) + (1 - comp_1_weight) * comp_2_dist.prob(w))
  
# Mixture prior parameters shared across DenseVariational layer instances
prior_params, prior_sigma = mixture_prior_params(sigma_1=1.0, sigma_2=0.1, pi=0.2)

In [4]:
# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def  posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential(
        [tfp.layers.VariableLayer(2 * n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n], scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
])

# Specify the prior over `keras.layers.Dense` `kernel` and `bias`.
def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t, scale=1),
            reinterpreted_batch_ndims=1)),
])

In [5]:
df = pd.read_csv('scaled_dummied_task.csv')
#df.head()

In [6]:
permuts = pd.read_csv('../HolyGrail/permutations.csv')

In [7]:
x_train = df.drop('Target', axis = 1)
y_train = df['Target']

In [8]:
n_epochs = 100000
lr = 5e-3
batch_size = x_train.shape[0]

In [9]:
def implement_wape(ys, preds):
    print(np.array(ys) - np.array(preds))
    numer8 = np.sum(np.abs(np.array(ys) - np.array(preds)))
    print(numer8)
    w = (numer8)/np.sum(np.abs(ys))
    return w

In [11]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import sys
train_size = len(x_train)
X = []
y = []
X_val = []
y_val = []

space = {'choice': hp.choice('num_layers',
                             [{'layers':'two', },
                              {'layers':'three', 
                               'units3': hp.choice('units3', [32, 64])
                              }
                             ]),
         'units1': hp.choice('units1', [16, 32, 64]),
         'units2': hp.choice('units2', [16, 32, 64]),
         'optimizer': hp.choice('optimizer',['adadelta','adam','rmsprop']),
        }

def f_nn(params):   

    print ('Params testing: ', params)
    model_in = Input(shape=(x_train.shape[1],))
    x = tfp.layers.DenseVariational(int('{}'.format(params['units1'])), posterior_mean_field, prior_trainable, kl_weight=1/train_size)(model_in)
    x = LeakyReLU(0.1)(x)
    x = tfp.layers.DenseVariational(int('{}'.format(params['units2'])), posterior_mean_field, prior_trainable, kl_weight=1/train_size)(x)
    x = LeakyReLU(0.1)(x)
    
    if params['choice']['layers']== 'three':
        x = tfp.layers.DenseVariational(int('{}'.format(params['choice']['units3'])), posterior_mean_field, prior_trainable, kl_weight=1/train_size)(x)
        x = LeakyReLU(0.1)(x) 
    
    model_out_loc = tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight=1/train_size)(x)
    model_out_scale = tfp.layers.DenseVariational(1, posterior_mean_field, prior_trainable, kl_weight=1/train_size)(x)
    model_out = DistributionLambda(lambda t: tfd.Normal(loc=t[0],
                                                        scale=1e-7 + tf.math.softplus(1e-3 * t[1])))([model_out_loc,
                                                                                                      model_out_scale])
    model = Model(model_in, model_out)
    model.compile(loss=neg_log_likelihood_with_dist, optimizer=params['optimizer'], metrics=['mse'])

    model.fit(x_train, y_train, epochs = n_epochs, batch_size = batch_size, verbose = 1)

    y_pred_list = []
    for i in range(len(x_train)):
        tensor = tf.constant([x_train.iloc[i]], dtype='float32')
        pred_list = []
        for j in range(500):
            y_pred = model.predict(tensor)
            pred_list.append(y_pred)
        y_pred_list.append(pred_list)

    y_means = np.mean(y_pred_list, axis = 1)
    y_means = y_means.reshape(len(x_train),)
    wape = implement_wape(y_means, y_train)
    print('WAPE: {}' .format(wape))
    return wape
    
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=5, trials=trials)
print('best: ', best)

('Params testing: ', {'units1': 32, 'units2': 32, 'optimizer': 'adadelta', 'choice': {'layers': 'two'}})
  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <type 'NoneType'>
Train on 600 samples                                 
Epoch 1/5                                            
600/600 [==============================]             
 - 1s 2ms/sample - loss: 807.1666 - mse: 763.0327    

Epoch 2/5                                            
600/600 [==============================]             
 - 0s 18us/sample - loss: 639.5879 - mse: 727.7476   

Epoch 3/5                                            
600/600 [==============================]             
 - 0s 19us/sample - loss: 1227.6436 - mse: 1113.7015 

Epoch 4/5                                            
600/600 [==============================]             
 - 0s 19us/sample - 

600/600 [==============================]                                   
 - 0s 18us/sample - loss: 326.9134 - mse: 323.0713                         

[8.64262379 5.09320325]                                                    
13.735827048041102                                                         
WAPE: 0.993665868618                                                       
100%|██████████| 5/5 [00:26<00:00,  5.20s/trial, best loss: 0.990106703395]
('best: ', {'units1': 1, 'units2': 1, 'optimizer': 0, 'num_layers': 0})


In [ ]:
y_pred_list = []
for i in range(len(permuts)):
    tensor = tf.constant([permuts.iloc[i]], dtype='float32')
    pred_list = []
    for j in range(5):
        y_pred = model.predict(tensor)
        pred_list.append(y_pred)
    y_pred_list.append(pred_list)

In [ ]:
y_means = np.mean(y_pred_list, axis = 1)
y_sigmas = np.std(y_pred_list, axis = 1)

In [ ]:
y_sigmas = y_sigmas.reshape(len(permuts),)
y_means = y_means.reshape(len(permuts),)

In [ ]:
idx = range(len(permuts))
plt.figure(figsize = (15, 7))
plt.plot(idx, y_means, 'r-', label='Predictive mean');
plt.fill_between(idx, y_means + 2 * y_sigmas,
                 y_means - 2 * y_sigmas, alpha=0.5)
plt.legend()
plt.title('Predictions');